<a href="https://colab.research.google.com/github/Veena7580/Homework/blob/main/midterm/midterm-2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2024 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 24th.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [69]:
def priest(sex, age, respirateRt, oxSaturation, hrtrate, bp, temperature, alertness, insOxygen, perfrmsts):
    """
    (String, float, float, float, float, float, float, String, String, String) -> float

    Calculates the PRIEST COVID-19 severity score and maps it to a 30-day risk percentage.

    :param sex: Sex assigned at birth ('Male' or 'Female')
    :param age: Age in years
    :param respirateRt: Respiratory rate in breaths per minute
    :param oxSaturation: Oxygen saturation as a percentage (0-1, e.g., 0.95 for 95%)
    :param hrtrate: Heart rate in beats per minute
    :param bp: Systolic blood pressure in mmHg
    :param temperature: Body temperature in degrees Celsius
    :param alertness: Alertness state ('Alert' or 'Confused or not alert')
    :param insOxygen: Inspired oxygen ('Air' or 'Supplemental oxygen')
    :param perfrmsts: Performance status (e.g., 'Unrestricted normal activity', 'Limited activity, can self-care', etc.)

    :return: Risk percentage between 0 and 1 or None for invalid input.

    >>> priest("FEMALE", 80, 23, 0.96, 105, 115, 38.1, "ALERT", "SUPPLEMENTAL OXYGEN", "Limited activity, can self-care")
    0.34
    >>> priest("MALE", 16, 10, 0.96, 105, 115, 38.1, "ALERT", "AIR", "Unrestricted normal activity")
    0.03
    >>> priest("FEMALE", 18, 23, 0.96, 135, 115, 38.1, "ALERT", "AIR", "Unrestricted normal activity")
    0.15
    """

    # Initialize score
    prscr = 0

    # Map inputs to lower case for flexibility in handling case variations
    sex = sex.lower()
    alertness = alertness.lower()
    insOxygen = insOxygen.lower()
    perfrmsts = perfrmsts.lower()

    # Validate inputs
    if sex not in ["male", "female"]:
        return None
    if insOxygen not in ["air", "supplemental oxygen"]:
        return None
    if alertness not in ["alert", "confused or not alert"]:
        return None

    # Sex scoring
    if sex == "male":
        prscr += 1

    # Age scoring
    if age >= 16 and age <= 49:
        prscr += 0
    elif age >= 50 and age <= 65:
        prscr += 2
    elif age >= 66 and age <= 80:
        prscr += 3
    elif age > 80:
        prscr += 4

    # Alertness scoring
    if alertness == "confused or not alert":
        prscr += 3

    # Performance status scoring
    if perfrmsts == "limited self-care":
        prscr += 3
    elif perfrmsts == "limited activity, can self-care":
        prscr += 2
    elif perfrmsts == "limited strenuous activity, can do light activity":
        prscr += 1
    elif perfrmsts == "bed/chair bound, no self-care":
        prscr += 4

    # Inspired oxygen scoring
    if insOxygen == "supplemental oxygen":
        prscr += 2

    # Respiratory rate scoring
    if respirateRt < 9:
        prscr += 3
    elif respirateRt > 24:
        prscr += 3
    elif respirateRt >= 21 and respirateRt <= 24:
        prscr += 2
    elif respirateRt >= 9 and respirateRt <= 11:
        prscr += 1

    # Oxygen saturation scoring
    if oxSaturation < 0.92:
        prscr += 3
    elif oxSaturation >= 0.92 and oxSaturation <= 0.93:
        prscr += 2
    elif oxSaturation >= 0.94 and oxSaturation <= 0.95:
        prscr += 1

    # Heart rate scoring
    if hrtrate > 130:
        prscr += 3
    elif hrtrate >= 111 and hrtrate <= 130:
        prscr += 2
    elif hrtrate >= 91 and hrtrate <= 110:
        prscr += 1
    elif hrtrate < 41:
        prscr += 3
    elif hrtrate >= 41 and hrtrate <= 50:
        prscr += 1

    # Systolic BP scoring
    if bp > 219:
        prscr += 3
    elif bp >= 101 and bp <= 110:
        prscr += 1
    elif bp >= 91 and bp <= 100:
        prscr += 2
    elif bp < 91:
        prscr += 3

    # Temperature scoring
    if temperature > 39.0:
        prscr += 2
    elif temperature >= 38.1 and temperature <= 39.0:
        prscr += 1
    elif temperature < 35.1:
        prscr += 3
    elif temperature >= 35.1 and temperature <= 36.0:
        prscr += 1

    # Convert PRIEST score to probability
    percscale = [0.01, 0.01, 0.02, 0.02, 0.03, 0.09, 0.15, 0.18, 0.22, 0.26, 0.29, 0.34, 0.38, 0.46, 0.47, 0.49, 0.55,
                 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.99]

    # Use the PRIEST score to map to the corresponding probability
    if prscr < 26:
        return percscale[prscr]

    return 0.99

In [70]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("FEMALE", 80, 23, 0.96, 105, 115, 38.1, "ALERT", "SUPPLEMENTAL OXYGEN", "Limited activity, can self-care")
Expecting:
    0.34
ok
Trying:
    priest("MALE", 16, 10, 0.96, 105, 115, 38.1, "ALERT", "AIR", "Unrestricted normal activity")
Expecting:
    0.03
ok
Trying:
    priest("FEMALE", 18, 23, 0.96, 135, 115, 38.1, "ALERT", "AIR", "Unrestricted normal activity")
Expecting:
    0.15
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [71]:
import requests

def find_hospital(age, sex, risk_pct):
    """
    Finds the hospital based on age, sex, and risk percentage by querying a REST API.

    Parameters:
    age (int): The patient's age.
    sex (str): The patient's sex ('male' or 'female').
    risk_pct (float): The patient's risk percentage as a decimal.

    Returns:
    str: The name of the hospital.
    None: If the request fails or invalid input is provided.

    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'
    >>> find_hospital(70, 'female', 0.3)
    'St. Joseph’s Hospital'
    >>> find_hospital(55, 'male', 0.99)
    'Grady Memorial Hospital'
    """
    try:
        url = f"https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk_pct}"
        response = requests.get(url)
        data = response.json()
        return data.get('hospital')
    except:
        return None


In [72]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(70, 'female', 0.3)
Expecting:
    'St. Joseph’s Hospital'
**********************************************************************
File "__main__", line 18, in NoName
Failed example:
    find_hospital(70, 'female', 0.3)
Expected:
    'St. Joseph’s Hospital'
Got:
    'Wesley Woods Geriatric Hospital'
Trying:
    find_hospital(55, 'male', 0.99)
Expecting:
    'Grady Memorial Hospital'
**********************************************************************
File "__main__", line 20, in NoName
Failed example:
    find_hospital(55, 'male', 0.99)
Expected:
    'Grady Memorial Hospital'
Got:
    'Emory Dunwoody Medical Center'


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [73]:
import requests

def get_address(hospital_name):
    """
    Retrieves the address of a hospital given its name.

    Parameters:
    hospital_name (str): The name of the hospital to look up.

    Returns:
    str or None: The address of the hospital if found, otherwise None.

    Example:
    >>> get_address("CHILDRENS HEALTHCARE OF ATLANTA AT SCOTTISH RITE")
    '1001 JOHNSON FERRY RD'

    >>> get_address("BUCKHEAD AMBULATORY SURGICAL CENTER")
    '2061 PEACHTREE RD NW, UNIT 300'

    >>> get_address("EMORY DUNWOODY MEDICAL CENTER")
    '4500 NORTH SHALLOWFORD ROAD'
    """
    url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"

    # Retrieve the JSON data
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
    except requests.RequestException as e:
        print(f"Error retrieving data: {e}")
        return None
    except ValueError:
        print("Error decoding JSON")
        return None

    # Search for the hospital by name
    hospital_info = data.get(hospital_name)

    # If found, return the address, otherwise return None
    return hospital_info.get("ADDRESS") if hospital_info else None

# Example usage:
# print(get_address("CHILDRENS HEALTHCARE OF ATLANTA AT SCOTTISH RITE"))

In [74]:
get_address("CHILDRENS HEALTHCARE OF ATLANTA AT SCOTTISH RITE")

'1001 JOHNSON FERRY RD'

In [75]:
get_address("BUCKHEAD AMBULATORY SURGICAL CENTER")

'2061 PEACHTREE RD NW, UNIT 300'

In [76]:
get_address("EMORY DUNWOODY MEDICAL CENTER")

'4500 NORTH SHALLOWFORD ROAD'

In [77]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("CHILDRENS HEALTHCARE OF ATLANTA AT SCOTTISH RITE")
Expecting:
    '1001 JOHNSON FERRY RD'
ok
Trying:
    get_address("BUCKHEAD AMBULATORY SURGICAL CENTER")
Expecting:
    '2061 PEACHTREE RD NW, UNIT 300'
ok
Trying:
    get_address("EMORY DUNWOODY MEDICAL CENTER")
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [78]:
import csv
import requests

def priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status):
    """
    PRIEST risk calculation placeholder.
    Replace this with actual calculation logic.
    """
    if o2sat < 95:
        return "High"
    elif breath > 20 or systolic < 100:
        return "Medium"
    else:
        return "Low"

def find_hospital(age, sex, risk):
    """
    Placeholder for finding a hospital based on age, sex, and risk.
    """
    if risk == "High":
        return "Emergency Hospital"
    elif risk == "Medium":
        return "General Hospital"
    else:
        return "Local Clinic"

def get_address(hospital):
    """
    Returns a placeholder address for a given hospital.
    """
    addresses = {
        "Emergency Hospital": "789 Main St, City",
        "General Hospital": "456 Health Ave, Town",
        "Local Clinic": "123 Wellness Rd, Village"
    }
    return addresses.get(hospital, "Unknown Address")

def process_people(url):
    """
    Downloads and processes the patient data from a given URL and returns a dictionary.

    The dictionary maps each patient ID to their medical information, including
    the calculated PRIEST risk, hospital, and hospital address.

    Parameters:
    - url (str): The URL to download the pipe-delimited (.psv) patient data file.

    Returns:
    - dict: A dictionary where the key is the patient ID and the value is a list
      with all the patient details.
    """
    # Download the file content from the URL
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful

    # Read the file content using the CSV reader with pipe delimiter
    lines = response.text.strip().split('\n')
    reader = csv.reader(lines, delimiter='|')

    # Skip the header row
    next(reader)

    patients_data = {}

    for row in reader:
        if len(row) < 11:
            continue  # Skip rows that don't have enough fields

        patient_id = row[0]
        sex = row[1]
        age = float(row[2])
        breath = float(row[3])
        o2sat = float(row[4])
        heart = float(row[5])
        systolic = float(row[6])
        temp = float(row[7])
        alertness = row[8]
        inspired = row[9]
        status = row[10]

        # Calculate the PRIEST risk
        risk = priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status)

        # Determine the hospital and its address
        hospital = find_hospital(age, sex, risk)
        address = get_address(hospital)

        # Store the patient data in the dictionary
        patients_data[patient_id] = [
            sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, risk, hospital, address
        ]

    return patients_data

# Example usage
url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
result = process_people(url)
for patient_id, details in result.items():
    print(f"Patient {patient_id}: {details}")

Patient E9559: ['FEMALE', 40.0, 24.0, 0.96, 105.0, 115.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City']
Patient E9385: ['Female', 51.0, 19.0, 0.99, 90.0, 91.0, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City']
Patient E3067: ['female', 40.0, 29.0, 0.96, 105.0, 95.0, 38.1, 'ALERT', 'air', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City']
Patient E9422: ['FEMALE', 66.0, 19.0, 0.96, 135.0, 115.0, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care', 'High', 'Emergency Hospital', '789 Main St, City']
Patient E8661: ['MALE', 18.0, 22.0, 0.95, 105.0, 115.0, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City']
Patient E6235: ['Male', 16.0, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'High', 'Emergency Hospital', '

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [79]:
import requests
import json

# Step 1: Load the provided results JSON from the given link
url = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
response = requests.get(url)
expected_results = response.json()

# step 2:Call the process_people function to get the generated results
file_url = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
generated_results = process_people(file_url)

print(generated_results)
print(expected_results)



{'E9559': ['FEMALE', 40.0, 24.0, 0.96, 105.0, 115.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City'], 'E9385': ['Female', 51.0, 19.0, 0.99, 90.0, 91.0, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City'], 'E3067': ['female', 40.0, 29.0, 0.96, 105.0, 95.0, 38.1, 'ALERT', 'air', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City'], 'E9422': ['FEMALE', 66.0, 19.0, 0.96, 135.0, 115.0, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care', 'High', 'Emergency Hospital', '789 Main St, City'], 'E8661': ['MALE', 18.0, 22.0, 0.95, 105.0, 115.0, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City'], 'E6235': ['Male', 16.0, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 'High', 'Emergency Hospital', '789 Main St, City'], 'E4451': 

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---